In [1]:
#pip install -U discord==1.7.3 


import nltk 
#import discord
nltk.download('punkt')

from nltk import word_tokenize,sent_tokenize

from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
#read more on the steamer https://towardsdatascience.com/stemming-lemmatization-what-ba782b7c0bd8
import numpy as np 
import tflearn
import tensorflow as tf
import random
import json
import pickle

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sharaftariq/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2022-12-16 19:31:22.661304: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
import pymongo


In [3]:
client = pymongo.MongoClient("mongodb://localhost:27017/")
 
# Database Name
db = client["admin"]
 
# Collection Name
rawtitle = db["rawtitle"]
 
bestmovieyr = db['bestmovieyr']

bestmovie = db['bestmovie']

bestshow = db['bestshow']

bestshowyr = db ['bestshowyr']

In [4]:
with open("intents.json") as file:
    data = json.load(file)

try:
    with open("data.pickle","rb") as f:
        words, labels, training, output = pickle.load(f)

except:
    words = []
    labels = []
    docs_x = []
    docs_y = []
    for intent in data["intents"]:
        for pattern in intent["patterns"]:
            wrds = nltk.word_tokenize(pattern)
            words.extend(wrds)
            docs_x.append(wrds)
            docs_y.append(intent["tag"])
            
        if intent["tag"] not in labels:
            labels.append(intent["tag"])


    words = [stemmer.stem(w.lower()) for w in words if w != "?"]
    words = sorted(list(set(words)))
    labels = sorted(labels)

    training = []
    output = []
    out_empty = [0 for _ in range(len(labels))]

    for x, doc in enumerate(docs_x):
        bag = []

        wrds = [stemmer.stem(w.lower()) for w in doc]

        for w in words:
            if w in wrds:
               bag.append(1)
            else:
              bag.append(0)
    
        output_row = out_empty[:]
        output_row[labels.index(docs_y[x])] = 1
        
        training.append(bag)
        output.append(output_row)

    training = np.array(training)
    output = np.array(output)
    
    with open("data.pickle","wb") as f:
        pickle.dump((words, labels, training, output), f)



net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)
model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
model.save("model.tflearn")

try:
    model.load("model.tflearn")
except:
    model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
    model.save("model.tflearn")


def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
    
    return np.array(bag)



def chat():
    print("Start talking with the bot! (type quit to stop)")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        result = model.predict([bag_of_words(inp, words)])[0]
        result_index = np.argmax(result)
        tag = labels[result_index]

        if result[result_index] > 0.7:
            for tg in data["intents"]:
                #if tg['tag'] == tag:
                if tg['tag'] == 'q1':
                    for x in bestmovieyr.find().sort("RELEASE_YEAR",-1).limit(1):
                         print(x['TITLE'])
                elif tg['tag'] == "q2":
                     for x in bestmovie.find().sort("SCORE",-1).limit(1):
                        print(x['TITLE'])
                elif tg['tag'] == "q3":
                    for x in bestshowyr.find().sort("RELEASE_YEAR",-1).limit(1):
                        print(x['TITLE'])
                elif tg['tag'] == "q4":
                    for x in bestshow.find().sort("SCORE",-1).limit(1):
                         print(x['TITLE'])
                elif tg['tag'] == "q5":
                    for x in bestshowyr.find({},{ "_id": 0, "TITLE": 1, "MAIN_PRODUCTION": 1 }).sort("MAIN_PRODUCTION",+1).limit(2):
                        print(x["TITLE"])
                elif tg['tag'] == "q6":
                    for x in rawtitle.find({},{"_id":0,"title":1, "release_year":1}).sort("release_year",+1).limit(1):
                         print(x)
                elif tg['tag'] == "q7":
                    for x in rawtitle.find({},{"_id":0, "title":1, "age_certification":1}).sort("age_certification",-1).limit(1):
                        print(x)
                elif tg['tag'] == "q8":
                    for x in bestmovieyr.find({},{"_id":0, "TITLE": 1,"MAIN_GENRE":1}).sort("MAIN_GENRE",+1).limit(1):
                        print(x["TITLE"])
                elif tg['tag'] == "q9":
                    for x in bestmovie.find({},{"_id":0,"TITLE":1,"SCORE":1}).sort("SCORE",-1).limit(5):
                        print(x)
                elif tg['tag'] == "q10":
                    for x in bestshow.find({},{"_id":0,"TITLE":1,"RELEASE_YEAR":1}).sort("RELEASE_YEAR",+1).limit(8):
                        print(x)

           # print(random.choice(responses))

        else:
            print("I didnt get that. Can you explain or try again.")
chat()


Training Step: 1999  | total loss: 0.00141 | time: 0.005s
| Adam | epoch: 1000 | loss: 0.00141 - acc: 1.0000 -- iter: 08/14
Training Step: 2000  | total loss: 0.00143 | time: 0.010s
| Adam | epoch: 1000 | loss: 0.00143 - acc: 1.0000 -- iter: 14/14
--
INFO:tensorflow:/Users/sharaftariq/Downloads/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:Restoring parameters from /Users/sharaftariq/Downloads/model.tflearn
Start talking with the bot! (type quit to stop)
You: hi
The Tinder Swindler
David Attenborough: A Life on Our Planet
Heartstopper
Breaking Bad
Anne with an E
Trailer Park Boys
{'title': 'Five Came Back: The Reference Films', 'release_year': 1945}
{'title': 'Pokémon', 'age_certification': 'TV-Y7'}
Dangal
{'TITLE': 'David Attenborough: A Life on Our Planet', 'SCORE': 9.0}
{'TITLE': 'Forrest Gump', 'SCORE': 8.8}
{'TITLE': 'Inception', 'SCORE': 8.8}
{'TITLE': 'Bo Burnham: Inside', 'SCORE': 8.7}
{'TITLE': 'Anbe Sivam', 'SCORE': 8.7}
{'TITLE': "Mo